# Preparation

```
pip install statsbombpy
pip install pyarrow
```

In [1]:
import os
import pandas as pd
from statsbombpy import sb
import warnings
warnings.filterwarnings("ignore", message="credentials were not supplied")

## Load

FIFA World Cup - South Korea vs. Portugal

In [2]:
df_events = sb.events(match_id=3857262)

In [3]:
df_events

,bad_behaviour_card,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 433, 'lineup': [{'player': {'id'...",South Korea,791,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 433, 'lineup': [{'player': {'id'...",Portugal,780,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Portugal,780,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,South Korea,791,00:00:00.000,Half Start,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,South Korea,791,00:00:00.000,Half Start,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Tactical,Ui-Jo Hwang,NaN,South Korea,791,00:36:13.753,Substitution,NaN
3375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Tactical,William Silva de Carvalho,NaN,Portugal,780,00:36:13.762,Substitution,NaN
3376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Tactical,Bernardo Mota Veiga de Carvalho e Silva,NaN,Portugal,780,00:36:13.790,Substitution,NaN
3377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Tactical,Yu-Min Cho,NaN,South Korea,791,00:47:46.061,Substitution,NaN


## Extract and Transform

### Fix Timestamp

In [4]:
df_events["timestamp"] = pd.to_timedelta(df_events.timestamp)

### Remove Events with No Location

In [5]:
df_pass = df_events[ ~pd.isna(df_events.location) ].copy()

### Start (X, Y)

In [6]:
df_pass[["x_start", "y_start"]] = pd.DataFrame(
    df_pass.location.to_list(),
    index=df_pass.index
)

In [7]:
df_pass.drop(columns="location", inplace=True)

### Pass Only

In [8]:
df_pass = df_pass[ (df_pass.type == "Pass") ]

### End (X, Y)

In [9]:
df_pass[["x_end", "y_end"]] = pd.DataFrame(
    df_pass.pass_end_location.to_list(),
    index=df_pass.index
)

In [10]:
df_pass.drop(columns="pass_end_location", inplace=True)

## Write to File

In [11]:
path_data = "./data"

In [12]:
os.makedirs(path_data, exist_ok=True)

### Parquet (Full)

In [13]:
df_pass.columns

Index(['bad_behaviour_card', 'ball_receipt_outcome', 'ball_recovery_offensive',
       'ball_recovery_recovery_failure', 'block_deflection',
       'carry_end_location', 'clearance_aerial_won', 'clearance_body_part',
       'clearance_head', 'clearance_left_foot', 'clearance_right_foot',
       'counterpress', 'dribble_nutmeg', 'dribble_outcome', 'dribble_overrun',
       'duel_outcome', 'duel_type', 'duration', 'foul_committed_card',
       'foul_committed_type', 'foul_won_defensive', 'goalkeeper_body_part',
       'goalkeeper_end_location', 'goalkeeper_outcome', 'goalkeeper_position',
       'goalkeeper_technique', 'goalkeeper_type', 'id', 'index',
       'injury_stoppage_in_chain', 'interception_outcome', 'match_id',
       'minute', 'miscontrol_aerial_won', 'off_camera', 'out',
       'pass_aerial_won', 'pass_angle', 'pass_assisted_shot_id',
       'pass_body_part', 'pass_cross', 'pass_cut_back', 'pass_deflected',
       'pass_goal_assist', 'pass_height', 'pass_inswinging', 'pass_l

In [14]:
df_pass.to_parquet(
    os.path.join(path_data, "events_pass_full.parquet"),
    index=False
)

### CSV (Simplified)

In [15]:
df_simple = df_pass.loc[:, ["period", "team", "type", "x_start", "y_start", "x_end", "y_end"]]
df_simple

,period,team,type,x_start,y_start,x_end,y_end
6,1,South Korea,Pass,60.0,40.0,39.9,29.0
7,1,South Korea,Pass,39.7,30.0,31.9,16.7
8,1,South Korea,Pass,37.4,8.9,49.8,3.6
9,1,South Korea,Pass,42.1,3.4,31.5,3.6
10,1,South Korea,Pass,30.6,4.1,57.7,11.7
...,...,...,...,...,...,...,...
1008,2,South Korea,Pass,94.3,76.4,89.5,78.9
1009,2,South Korea,Pass,88.1,78.2,79.6,76.2
1010,2,South Korea,Pass,79.3,75.4,81.9,72.3
1011,2,Portugal,Pass,38.2,7.8,68.1,12.9


In [16]:
df_simple.to_csv(
    os.path.join(path_data, "events_pass_simple.csv"),
    index=False
)